In [1]:


### parameters
place = 'tel_aviv'


In [2]:
import osmnx as ox
from osmnx import io
import geopandas as gpd
import pandas as pd
import os
from pathlib import Path
import warnings
warnings.filterwarnings(action='ignore')
crs_prj = 'EPSG:2039'



# Get the current working directory (e.g., the folder you're running from)
cwd = Path().resolve()

# Get the parent directory
parent_folder = f'{cwd.parent}/places/{place}'
data_folder = f'{parent_folder}/shp'
os.makedirs(f'{parent_folder}',exist_ok=True)
os.makedirs(f'{parent_folder}/shp',exist_ok=True)


In [137]:
# Download OSM graph for the place
place = 'tel aviv, Israel'
graph = ox.graph_from_place(place, network_type='all')
print('✔️ Finished downloading OSM data')


# Project the graph to a specific CRS for analysis
graph_pro = ox.projection.project_graph(graph, to_crs=crs_prj)

# Save the projected graph to a GeoPackage file
io.save_graph_geopackage(
    graph_pro,
    filepath=f'{data_folder}/osm_data.gpkg',
    encoding='utf-8',
    directed=False
)

In [7]:
# Load your two layers
# Layer 1: your own street edges
street_edges = gpd.read_file(f"{data_folder}/streets.shp").reset_index(drop=True)
if 'highway' in street_edges.columns:
    street_edges.drop(columns=['highway'], inplace=True)
if 'overlap_area' in street_edges.columns:
    street_edges.drop(columns=['overlap_area'], inplace=True)
# Layer 2: OSM street edges with a 'highway' `field
osm_edges = gpd.read_file(f'{data_folder}/osm_data.gpkg',layer = 'edges' )[['geometry', 'highway']].reset_index()
# street_edges.drop(columns=['overlay_hi','overlay_ar','matched_hi'], inplace=True)


In [8]:

# Step 1: Buffer the street and OSM layers
buffered_edges = street_edges.copy()
buffered_edges['geometry'] = buffered_edges.geometry.buffer(10)

buffered_osm = osm_edges.copy()
buffered_osm['geometry'] = buffered_osm.geometry.buffer(0.5)

# Step 2: Intersect buffered layers and compute area
intersections = gpd.overlay(buffered_edges, buffered_osm, how='intersection')
intersections['overlap_area'] = intersections.geometry.area


# Step 3: Build lookup for original geometries
# Needed to calculate real distances later
orig_geoms = street_edges.set_index('oidrechov')['geometry'].to_dict()

osm_geoms = osm_edges[['geometry', 'highway']].copy()
osm_geoms['osm_index'] = osm_geoms.index


# Step 4: Decision function per `oidrechov`
def choose_highway(oid, w_overlap=0.7, w_distance=0.3):
    if oid not in orig_geoms:
        return 'not known'

    original_geom = orig_geoms[oid]

    # Get all intersections for this oidrechov
    group_overlaps = intersections[intersections['oidrechov'] == oid]
    if group_overlaps.empty:
        return 'not known'

    # Filter overlaps < 5m²
    group_overlaps = group_overlaps[group_overlaps['overlap_area'] >= 10]
    if group_overlaps.empty:
        return 'not known'

    # Group by highway, sum area and find min distance per highway
    candidates = []
    for highway, group in group_overlaps.groupby('highway'):
        total_area = group['overlap_area'].sum()
        min_dist = group['index'].apply(
            lambda idx: original_geom.distance(osm_geoms.loc[idx, 'geometry'])
        ).min()
        candidates.append((highway, total_area, min_dist))

    # Create DataFrame of candidates
    df = pd.DataFrame(candidates, columns=['highway', 'total_overlap', 'min_distance'])

    # Normalize values
    df['norm_overlap'] = (df['total_overlap'] - df['total_overlap'].min()) / (df['total_overlap'].max() - df['total_overlap'].min() + 1e-6)
    df['norm_distance'] = (df['min_distance'] - df['min_distance'].min()) / (df['min_distance'].max() - df['min_distance'].min() + 1e-6)

    # Score based on weighted combination
    df['score'] = w_overlap * df['norm_overlap'] - w_distance * df['norm_distance']

    # Return highway with highest score
    best = df.sort_values('score', ascending=False).iloc[0]
    return best['highway']


# Step 5: Apply decision to each oidrechov
unique_oids = street_edges['oidrechov']
highway_map = {oid: choose_highway(oid) for oid in unique_oids}

# Step 6: Assign result back to street_edges
street_edges['highway'] = street_edges['oidrechov'].map(highway_map)

# Step 7: Save results
street_edges.to_file(f"{data_folder}/streets.shp")